In [ ]:
import os
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms, datasets
print('hi')

In [ ]:
# 파라미터 설정하기
lr = 1e-3
batch_size = 64
num_epoch = 10

# 학습가중치와 텐서보드 로그저장
ckpt_dir = './checkpoint'
log_dir = './log'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# 네트워크 구축
class Net(nn.Module):
    def __init__(self):
        # 넷의 부모클래스에서 속성 상속받아옴
        super(Net, self).__init__()
        
        # 네트워크 구축에 필요한 레이어 초기화 
        self.conv1 = nn.Conv2d(1, 10, 5, 1, 0, 0)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.relu1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(10, 20, 5, 1, 0, 0)
        self.drop2 = nn.Dropout2d(p=0.5)
        self.pool2 = nn.MaxPool2d(2)
        self.relu2 = nn.ReLU()
        
        self.fc1 = nn.Linear(320, 50)
        self.relu1_fc1 = nn.ReLU()
        self.drop1_fc1 = nn.Dropout2d(0.5)
        
        self.fc1 = nn.Linear(50, 10)
        
    # 위의 초기화된 레이어를 연결하여 네트워크를 구축하였다
    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.relu1(x)
        
        x = self.conv2(x)
        x = self.drop2(x)
        x = self.pool2(x)
        x = self.relu2(x)
        
        # 텐서의 1차원은 자동, 2차원은 320 크기로
        # 개수가 6400개이면 앞은 20이 되어야함 그런데 -1하면 알아서 맞춰줌
        x = x.view(-1, 320)
        
        x = self.fc1(x)
        x = self.relu1_fc1(x)
        x = self.drop1_fc1(x)
        
        x = self.fc2(x)
        
        return x

In [ ]:
# 네트워크를 저장 또는 불러오는 함수
def save(ckpt_dir, net, optim, epoch):
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)
        torch.save({'net': net.state_dict(), 'optim': optim.state_dict()},
                   './%s/model_epoch%d.pth' % (ckpt_dir, epoch))
        
def load(ckpt_dir, net, optim):
    #경로의 모든파일 가져오기
    ckpt_lst = os.listdir(ckpt_dir)
    #알파벳순으로 정렬 마지막이 최근
    ckpt_lst.sort()
    
    # 파이토치객체를 최근것을 불러와서 dict형태로 저장
    dict_model = torch.load('./%s/%s' % (ckpt_dir, ckpt_lst[-1]))
    
    #네트워크의 파라미터(가중치)를 가져온것으로 업데이트
    net.load_state_dict(dict_model['net'])
    # 옵티마이저도 동일하게
    optim.load_state_dict(dict_model['optim'])
    return net, optim


In [ ]:
## mnist데이터 불러오기
# 전처리과정을 결합(compose)한다
#이미지데이터를 텐서데이터로 바꾸고, 0~1로 정규화
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])
#데이터셋이 존재하지않으면 다운로드, 라벨데이터도 같이
dataset = datasets.MNIST(download=True, root='./', train=True, transform=transform)

#데이터셋을 배치단위로 가져온다,
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)

# 데이터셋 데이터의 전체개수
num_data = len(loader.dataset)
# 전체데이터를 배치단위로 나누어 전체배치수 계산
num_batch = np.ceil(num_data / batch_size)


In [ ]:
## 네트워크 설정 및 필요한 손실함수 구현
# 사용자 정의 신경망이다. device는 cpu또는 gpu
net = Net().to(device)
#신경망의 가중치와 편향, 훈련하면서 최적화됨
params = net.parameters()

# 손실함수, 모델예측과 실제의 차이확인
fn_loss = nn.CrossEntropyLoss().to(device)
#소프트맥수 함수를 통과시켜 각 클래스에 대한 확률분포로 변환
fn_pred = lambda output: torch.softmax(output, dim=1)
# 예측정확도 계산, 확률이 최대인것을 찾고 답이 같으면 평균을 취해서 정확도 산출
fn_acc = lambda pred, label: ((pred.max(dim=1)[1] == label).type(torch.float)).mean()
# adam 최적화알고리즘으로 파라미터를 lr에 따라업데이트
optim = torch.optim.Adam(params, lr=lr)
# 텐서보드에 손실, 정확도등의 그래프 확인
writer = SummaryWriter(log_dir=log_dir)

In [ ]:
## 네트워크 학습 시작
#여러번 학습을 한다, 각 반복을 에폭이라한다
for epoch in range(1, num_epoch+1):
    #신경만을 훈련 모드로 설정=>dropout,BN은 훈련시에만 활성화해주어야한다
    net.train()
    #각 배치마다의 값을저장 저장할 리스트
    loss_arr = []
    acc_arr = []
    
    # 데이터로더에서 배치단위, X,Y,데이터 가져오기
    for batch, (input, label) in enumerate(loader, 1):
        # 장치에 데이터를 넣어서 계산준비
        input = input.to(device)
        label = label.to(device)
        #순전파 신경망에 데이터를 전달해서 예측출력
        output = net(input)
        # 신경망의 출력으로 예측값 계산
        pred = fn_pred(output)
        # 그래디언트 초기화, 각 배치마다 새로운 그래디언트를 계산하기 위해
        optim.zero_grad()
        #예측과 레이블을 비교해서 손실계산
        loss = fn_loss(output, label)
        # 정확도도 동일
        acc = fn_acc(pred, label)
        # 각 가중치에 대한 손실함수의 그래디언트 계산 => 그래디언트를 보고 가중치를 늘리거나 줄이기를 결정
        loss.backward()
        # 옵티마이저로 계산한 그래디언트를가지고 가중치 업데이트
        optim.step()
        #지금 배치의 손실과 정확도 저장
        loss_arr += [loss.item()]
        acc_arr += [acc.item()]
        #진행상황 출력
        print('TRAIN: EPOCH %04d/%04d | BATCH %04d/$04d | LOSS: %.4f | ACC: %.4f' %
              (epoch, num_epoch, num_batch, np.mean(loss_arr), np.mean(acc_arr)))
        
    #텐서보드 기록용
    writer.add_scalar('loss', np.mean(loss_arr), epoch)
    writer.add_scalar('acc', np.mean(acc_arr), epoch)
    
    #체크포인트 저장
    save(ckpt_dir=ckpt_dir, net=net, optim=optim, epoch=epoch)
#텐서보드 로깅을 마치고 객체닫기
writer.close()

